<a href="https://colab.research.google.com/github/i-huzaifa-arshad/InvokeAi-Colab/blob/Alpha/InvokeAi_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

<h1><center>Colab Notebook for InvokeAi v3.4.0</center></h1>
<h3><center>In case if you want to support my work 😉</center></h3>
<center>
    <a href="https://www.patreon.com/huzaifaarshad">Patreon</a>
    &nbsp; | &nbsp;
    <a href="https://www.buymeacoffee.com/huzaifaarshad">Buy me Coffee</a>
</center>


---




In [ ]:
#@markdown # **1. Connect Google Drive**
#@markdown ---

#@markdown - If you have low storage in drive, skip this step and proceed to install InvokeAi in Colab Storage
#@markdown - Please note once you restart runtime, you will lose all data, so download important files before closing the notebook

from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
#@markdown # **2. Install / Update InvokeAi**
#@markdown ---
#@markdown - It will clone the repository into `InvokeAI` folder automatically in your drive, so no need to create a separate folder

from IPython.display import clear_output

%cd /content/gdrive/MyDrive

# Clone the repository
!git clone https://github.com/i-huzaifa-arshad/InvokeAI
clear_output()

print('\u2714 Done!')

In [ ]:
#@markdown # **3. Install Dependencies (Required)**
#@markdown ---
#@markdown - It takes about ~5 minutes to install the required dependencies
#@markdown - In case you think it's not working, don't worry. Sit back and relax 😉

import warnings
warnings.filterwarnings("ignore")

%cd /content/gdrive/MyDrive/InvokeAI

print('[1;32mInstalling dependencies...')

# Installing requirements from Dependencies folder

!pip install -q --force-reinstall -r Dependencies/Requirements.txt > nul 2>&1
!pip install -q -e . > nul 2>&1

print ('\u2714 Done')

In [ ]:
#@markdown # **4-a. Model Download**
#@markdown ---

#@markdown - Installing stable diffusion models (SD 1.5, SDXL, SDXL Refiner). If you want to install ControlNet and other models, click on the below path
#@markdown - `/content/gdrive/MyDrive/InvokeAI/invokeai/configs/INITIAL_MODELS.yaml` and change `recommended: False` to `recommended: True`
#@markdown - Installing these default models will take about ~20 minutes. If you selected other models as well, it will take more time due to their size
#@markdown - If you get `AttributeError: 'set' object has no attribute 'extend'` error, just delete the `GUI` folder from your drive and run the cell again

import os
import warnings

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings("ignore")

%cd /content/gdrive/MyDrive/InvokeAI/

print('[1;32mDownloading models...')

!python /content/gdrive/MyDrive/InvokeAI/scripts/invokeai-model-install.py --root_dir /content/gdrive/MyDrive/InvokeAI/GUI --yes > /dev/null 2>&1

print ('\u2714 Done')


In [ ]:
#@markdown # **4-b. Download Custom model (Optional)**
#@markdown ---

#@markdown - Download a trained dreambooth `.ckpt model` or other desired models from `google drive` or `civitai.com`
#@markdown - Just write desired `model name` and `model url` and it will automatically download the models in the required path
#@markdown - You don't have to write model extension in `model_name` as it will automatically add it and save models like `model-name.ckpt` or `model-name.safetensors`
#@markdown - If you get some error about model while running the InvokeAI, install the model using `model manager` inside InvokeAI Webui

import os
from urllib.parse import urlparse

# Ask the user for the model name
model_name = "" # @param {type:"string"}

model_link = "" # @param {type:"string"}

# Define the directory where the model will be saved
model_dir = '/content/gdrive/MyDrive/InvokeAI/GUI/models/sd-1/main'

# Create the directory if it does not exist
os.makedirs(model_dir, exist_ok=True)

# Check if the URL is from Google Drive
if "drive.google.com" in model_link:
    # Add the .ckpt extension
    model_filename = f"{model_name}.ckpt"

    # Extract the file ID from the URL
    file_id = model_link.split('/')[-2]

    # Download the file from Google Drive
    !gdown https://drive.google.com/uc?id={file_id} -O {os.path.join(model_dir, model_filename)}

else:
    # Add the .safetensors extension
    model_filename = f"{model_name}.safetensors"

    # Download the model
    !wget -O {os.path.join(model_dir, model_filename)} "{model_link}"

clear_output()
print('Model Downloaded! 🚀')


In [ ]:
#@markdown # **5. Run InvokeAI**
#@markdown - Launching InvokeAI Web UI via SSH Tunnel
#@markdown - The outputs will be saved in the `Outputs` folder under `/content/gdrive/MyDrive/InvokeAI/GUI` path
#@markdown ---

import os
import shlex
import subprocess
from IPython.display import clear_output
from pathlib import Path
from typing import Union
clear_output()

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

id_rsa_file = "/root/.ssh/id_rsa"
id_rsa_pub_file = "/root/.ssh/id_rsa.pub"
if os.path.exists(id_rsa_file):
    os.remove(id_rsa_file)
if os.path.exists(id_rsa_pub_file):
    os.remove(id_rsa_pub_file)
clear_output()

def gen_key(path: Union[str, Path]) -> None:
    path = Path(path)
    arg_string = f'ssh-keygen -t rsa -b 4096 -N "" -q -f {path.as_posix()}'
    args = shlex.split(arg_string)
    subprocess.run(args, check=True)
    path.chmod(0o600)

ssh_name = "id_rsa"
ssh_path = Path("/root/.ssh/") / ssh_name
gen_key(ssh_path)
clear_output()

import threading
def tunnel():
  !ssh -R 80:127.0.0.1:9090 -o StrictHostKeyChecking=no -i /root/.ssh/id_rsa remote.moe
threading.Thread(target=tunnel, daemon=True).start()

%cd /content/gdrive/MyDrive/InvokeAI/
!python /content/gdrive/MyDrive/InvokeAI/scripts/invokeai-web.py --root /content/gdrive/MyDrive/InvokeAI/GUI